# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [40]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [41]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [42]:
# target column to numeric
spaceship['Transported'] = spaceship['Transported'].map({True: 1, False: 0})

# Fill boolean columns
spaceship['CryoSleep'] = spaceship['CryoSleep'].fillna(spaceship['CryoSleep'].mode()[0]).astype(bool)
spaceship['VIP'] = spaceship['VIP'].fillna(spaceship['VIP'].mode()[0]).astype(bool)

# Fill mode imputation
spaceship['HomePlanet'] = spaceship['HomePlanet'].fillna(spaceship['HomePlanet'].mode()[0])
spaceship['Destination'] = spaceship['Destination'].fillna(spaceship['Destination'].mode()[0])

# Handle Cabin
spaceship[['Deck', 'CabinNum', 'Side']] = spaceship['Cabin'].str.split('/', expand=True)
spaceship['Deck'] = spaceship['Deck'].fillna('U') 
spaceship['CabinNum'] = spaceship['CabinNum'].fillna('0').astype(str)
spaceship['Side'] = spaceship['Side'].fillna('U')  
spaceship.drop(columns=['Cabin'], inplace=True)

# Fill numerical fields with 0
num_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
spaceship[num_cols] = spaceship[num_cols].fillna(0)

# Fill Age with median
spaceship['Age'] = spaceship['Age'].fillna(spaceship['Age'].median())

#  Drop Name
spaceship.drop(columns=['Name'], inplace=True)


C:\Users\49151\AppData\Local\Temp\ipykernel_23384\3583390986.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  spaceship['CryoSleep'] = spaceship['CryoSleep'].fillna(spaceship['CryoSleep'].mode()[0]).astype(bool)
C:\Users\49151\AppData\Local\Temp\ipykernel_23384\3583390986.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  spaceship['VIP'] = spaceship['VIP'].fillna(spaceship['VIP'].mode()[0]).astype(bool)


In [43]:
spaceship['TotalSpend'] = spaceship[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
# group ID
spaceship['Group'] = spaceship['PassengerId'].str.split('_').str[0]

# Calculate group size
group_sizes = spaceship.groupby('Group')['PassengerId'].transform('count')
spaceship['GroupSize'] = group_sizes
spaceship['IsAlone'] = spaceship['GroupSize'] == 1

spaceship['AgeGroup'] = pd.cut(
    spaceship['Age'],
    bins=[0, 12, 18, 30, 50, 80],
    labels=['Child', 'Teen', 'YoungAdult', 'Adult', 'Senior']
)

for col in ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
    spaceship[f'Used_{col}'] = spaceship[col] > 0

spaceship['Cryo_VIP'] = spaceship['CryoSleep'].astype(int) + spaceship['VIP'].astype(int)

from sklearn.preprocessing import MinMaxScaler

num_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'TotalSpend', 'CabinNum', 'GroupSize']
scaler = MinMaxScaler()
spaceship[num_cols] = scaler.fit_transform(spaceship[num_cols])

spaceship = pd.get_dummies(spaceship, columns=['HomePlanet', 'Destination', 'Deck', 'Side', 'AgeGroup'], drop_first=True)


In [44]:
# target and features
X = spaceship.drop(columns=['Transported', 'PassengerId', 'Group']) 
y = spaceship['Transported']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [45]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Define base estimator
base_tree = DecisionTreeClassifier(random_state=42)

# Use BaggingClassifier with bootstrap=False → Pasting
best_model = BaggingClassifier(
    estimator=base_tree,
    bootstrap=False,  # this is key for Pasting
    random_state=42
)


- Evaluate your model

In [46]:
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report

print("Default Pasting Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Default Pasting Accuracy: 0.7492811960897068
              precision    recall  f1-score   support

           0       0.74      0.75      0.75       863
           1       0.75      0.75      0.75       876

    accuracy                           0.75      1739
   macro avg       0.75      0.75      0.75      1739
weighted avg       0.75      0.75      0.75      1739



**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [48]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_samples': [0.6, 0.8, 1.0],
    'estimator__max_depth': [None, 5, 10],
    'estimator__min_samples_split': [2, 5, 10]
}

- Run Grid Search

In [49]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    estimator=best_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5,
             estimator=BaggingClassifier(bootstrap=False,
                                         estimator=DecisionTreeClassifier(random_state=42),
                                         random_state=42),
             n_jobs=-1,
             param_grid={'estimator__max_depth': [None, 5, 10],
                         'estimator__min_samples_split': [2, 5, 10],
                         'max_samples': [0.6, 0.8, 1.0],
                         'n_estimators': [50, 100, 150]},
             scoring='accuracy', verbose=1)

- Evaluate your model

In [50]:
tuned_model = grid_search.best_estimator_
y_pred_tuned = tuned_model.predict(X_test)

print("Best Hyperparameters:", grid_search.best_params_)
print("Tuned Pasting Accuracy:", accuracy_score(y_test, y_pred_tuned))
print(classification_report(y_test, y_pred_tuned))


Best Hyperparameters: {'estimator__max_depth': None, 'estimator__min_samples_split': 10, 'max_samples': 0.6, 'n_estimators': 150}
Tuned Pasting Accuracy: 0.8073605520414031
              precision    recall  f1-score   support

           0       0.79      0.84      0.81       863
           1       0.83      0.78      0.80       876

    accuracy                           0.81      1739
   macro avg       0.81      0.81      0.81      1739
weighted avg       0.81      0.81      0.81      1739

